In [26]:
import numpy as np
import scipy as sp
from scipy import linalg as sp_linalg
from sinkhorn_knopp import sinkhorn_knopp as skp

V = 100

rng = np.random.RandomState(0)
P_x_given_y = np.triu(rng.uniform(low=0.1, high=0.9, size=[V, V]), k=1) # 0's on the diagonal
P_x_given_y += P_x_given_y.T # P(x|y) = P(y|x)

sk = skp.SinkhornKnopp()
P_x_given_y = sk.fit(P_x_given_y) #Produces approximately doubly stochastic P(x|y)=P(y|x) matrix
print(P_x_given_y.sum(axis=1)[:5], P_x_given_y.sum(axis=0)[:5])

p_i = rng.power(a=1.0, size=V)
p_i /= p_i.sum() # p_i should sum to 1
assert p_i.sum() == 1
P_i = np.diag(p_i)
P_i_inv = np.diag(1.0 / p_i)

similarities = P_x_given_y.dot(P_i_inv)
pmis = np.log(similarities)
P_ij = P_i.dot(similarities).dot(P_i) # P_ij should sum to 1
assert P_ij.sum() == 1

[1. 1. 1. 1. 1.] [0.99999114 1.00002225 0.99999797 0.99998379 0.99999218]


/Users/i.lobov/research/lib/python3.6/site-packages/ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in log
